In [34]:
import pandas as pd
import numpy as np

import librosa, librosa.display
import matplotlib.pyplot as plt

import matplotlib.axes as axes
from pydub import AudioSegment
# from pydub.playback import play
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import  Dense,Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [35]:
data = pd.read_csv('data.csv')

In [44]:
sample=data['File'].tolist()
sample=sample[:486]
output=data['Output'].tolist()
# output=output[:486]
print(len(output))
print(len(sample))
sample_cp=sample



488
486


In [66]:
def compute_spectrogram(file_path):
    signal, sample_rate = librosa.load(file_path, sr=10000)
    signal=signal[:10*22050]
    h_l = 512
    nfft = 2048
    stft = librosa.stft(y=signal, n_fft=nfft, hop_length=h_l)
    log_stft = librosa.amplitude_to_db(np.abs(stft))
    return log_stft

csv_file_path = 'data.csv'  # Path to your CSV file
column_name = 'File'  # Name of the column containing file paths

df = pd.read_csv(csv_file_path)
file_paths = df[column_name].tolist()
file_paths=file_paths[:485]

array_of_spectrograms = []
for file_path in file_paths:
    spectrogram = compute_spectrogram(file_path)
    array_of_spectrograms.append(spectrogram)

# Print the shapes of the spectrograms
# for spectrogram in array_of_spectrograms:
#     print(spectrogram)
print(array_of_spectrograms[0].shape)
print(array_of_spectrograms[0])

(1025, 196)
[[  8.432526   15.75954    19.04251   ...  17.037312   17.587282
   13.485317 ]
 [  6.1101203   9.7918215  16.026361  ...  11.920601   13.011922
   10.4215145]
 [  2.3528895   5.3675885   8.581322  ...   1.5018446  -5.2595654
   -2.6629143]
 ...
 [-45.457466  -50.211086  -55.741203  ... -55.741203  -55.741203
  -46.653004 ]
 [-45.476604  -52.62461   -55.741203  ... -55.741203  -55.741203
  -46.911617 ]
 [-45.469406  -50.3336    -55.741203  ... -55.741203  -55.741203
  -46.94143  ]]


In [67]:
np_array_of_stft = np.array(array_of_spectrograms)

np_array_of_stft.shape

(485, 1025, 196)

In [68]:
array_of_log_spectrograms = []
for spectrogram in array_of_spectrograms:
    # Normalize the spectrogram
    normalized_spectrogram = (spectrogram - np.min(spectrogram)) / (np.max(spectrogram) - np.min(spectrogram))

    # Apply the log transformation
    log_spectrogram = np.log(normalized_spectrogram + 1e-10)  # Adding a small value to avoid log(0) error

    array_of_log_spectrograms.append(log_spectrogram)


In [69]:
np_array_of_stft_log = np.array(array_of_log_spectrograms)
np_array_of_stft_log.shape

(485, 1025, 196)

In [70]:
labels=np.array(output)
labels=labels[:485]
# reshaped_spectrograms = np_array_of_stft_log.reshape((np_array_of_stft_log.shape[0], -1))
print(labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]


In [79]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
X_train, X_test, y_train, y_test = train_test_split(np_array_of_stft_log, labels, test_size=0.2, random_state=42)

# Create the CNN model
model = Sequential([
     Conv2D(32, (3, 3), activation='relu', input_shape=(np_array_of_stft_log.shape[1], np_array_of_stft_log.shape[2], 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    # Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

Epoch 1/10
13/13 [==============================] - 39s 3s/step - loss: 499.2314 - accuracy: 0.5412 - val_loss: 13.3228 - val_accuracy: 0.7732
Epoch 2/10
13/13 [==============================] - 35s 3s/step - loss: 5.3252 - accuracy: 0.8325 - val_loss: 6.9204 - val_accuracy: 0.7732
Epoch 3/10
13/13 [==============================] - 34s 3s/step - loss: 1.1874 - accuracy: 0.8969 - val_loss: 1.4928 - val_accuracy: 0.7526
Epoch 4/10
13/13 [==============================] - 34s 3s/step - loss: 0.3537 - accuracy: 0.9098 - val_loss: 0.9540 - val_accuracy: 0.8454
Epoch 5/10
13/13 [==============================] - 34s 3s/step - loss: 0.0719 - accuracy: 0.9716 - val_loss: 0.4122 - val_accuracy: 0.8557
Epoch 6/10
13/13 [==============================] - 35s 3s/step - loss: 0.0737 - accuracy: 0.9691 - val_loss: 0.2424 - val_accuracy: 0.9175
Epoch 7/10
13/13 [==============================] - 35s 3s/step - loss: 0.0498 - accuracy: 0.9820 - val_loss: 0.3049 - val_accuracy: 0.9175
Epoch 8/10
13/13 

In [80]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions (0 or 1)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred)

print("F1 Score:", f1)

4/4 [==============================] - 2s 389ms/step
F1 Score: 0.8648648648648649


In [161]:

from scipy.signal import butter, sosfilt
file_path="Last_13.wav"
signal, sample_rate = librosa.load(file_path, sr=10000)
signal=signal[:10*10000]
# sos = signal.butter(4, [4, 14.0], btype='band',fs=10000, output='sos')
# signal = signal.sosfilt(sos,signal)
# librosa.display.waveshow(signal, sr=10000)
# plt.show()
h_l = 512
nfft = 2048
stft = librosa.stft(y=signal, n_fft=nfft, hop_length=h_l)
log_stft = librosa.amplitude_to_db(np.abs(stft))


In [162]:
normalized_spectrogram = (log_stft - np.min(log_stft)) / (np.max(log_stft) - np.min(log_stft))

# Apply the log transformation
log_spectrogram = np.log(normalized_spectrogram + 1e-10) 
print(log_spectrogram)
# librosa.display.specshow(log_spectrogram, sr=10000, hop_length=h_l, x_axis='time', y_axis='linear')
# plt.show()
log_spectogram=np.reshape(log_spectrogram,(1,log_spectrogram.shape[0],log_spectrogram.shape[1],1))
prediction=model.predict(log_spectogram)
# ser1.write(prediction[0])
print(prediction)

[[-3.2388133e-01 -2.9804686e-01 -5.6377226e-01 ... -2.0979067e-02
  -9.5694110e-02 -5.3819078e-01]
 [-2.5471333e-01 -1.7826481e-01 -1.5001188e-01 ... -2.1074207e-02
  -9.9665076e-02 -2.9231602e-01]
 [-2.3572132e-01 -1.6179506e-01 -1.6712260e-01 ... -8.4300764e-02
  -2.0194900e-01 -3.1334269e-01]
 ...
 [-2.9171603e+00 -2.3025850e+01 -2.3025850e+01 ... -2.3025850e+01
  -2.3025850e+01 -2.3025850e+01]
 [-2.9115188e+00 -2.3025850e+01 -2.3025850e+01 ... -2.3025850e+01
  -2.3025850e+01 -2.3025850e+01]
 [-2.9172783e+00 -2.3025850e+01 -2.3025850e+01 ... -2.3025850e+01
  -2.3025850e+01 -2.3025850e+01]]
1/1 [==============================] - 0s 95ms/step
[[0.7744027]]


In [163]:
if prediction>0.5:
    prediction= 1
else:
    prediction= 0
    
prediction

1

In [111]:
model.save("Best_of_all_2")

INFO:tensorflow:Assets written to: Best_of_all\assets


INFO:tensorflow:Assets written to: Best_of_all\assets


In [122]:
new_model=tf.keras.models.load_model("Best_of_all")

In [123]:
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 1023, 194, 32)     320       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 511, 97, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 509, 95, 64)       18496     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 254, 47, 64)      0         
 g2D)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 764032)            0         
                                                                 
 dense_14 (Dense)            (None, 128)              